## Deep MLP using a Chamfer Loss for Jet-Images using jet constituents in HLS data
## Author: Andre Sznajder

In [1]:
#!fusermount -u drive
#! pip install einops

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#data_dir = '/content/gdrive/My Drive/Colab Notebooks/Data/'
#data_dir = '/Users/sznajder/cernbox/WorkM1/tensorflow_macos/arm64/workdir/data/hls4ml_LHCjet_30p_val/'

# Load train and test JetID datasets as numpy arrays 

In [3]:
import numpy as np
import h5py
import os
from einops import rearrange, reduce, repeat


#Data PATH
DATA_PATH = '/Users/sznajder/WorkM1/workdir/data/'

nconstit = 32

X_train_val = np.load("../../data/X_train_val_nconst_{}.npy".format(nconstit))
X_test = np.load("../../data/X_test_nconst_{}.npy".format(nconstit))
Y_train_val = np.load("../../data/Y_train_val_nconst_{}.npy".format(nconstit))
Y_test = np.load("../../data/Y_test_nconst_{}.npy".format(nconstit))

print("Loaded X_train_val ----> shape:", X_train_val.shape)
print("Loaded X_test      ----> shape:", X_test.shape)
print("Loaded Y_train_val ----> shape:", X_train_val.shape)
print("Loaded Y_test      ----> shape:", X_test.shape)

nfeat = X_train_val.shape[-1]

# Flatten the 2D ( eta x phi ) jet image into 1D array to input into a MLP 
X_train_val = rearrange(  X_train_val , 'batch nconstit nfeat -> batch (nconstit nfeat)' )  
X_test = rearrange(  X_test , 'batch nconstit nfeat -> batch (nconstit nfeat)' )  

print("Flattened X_train_val ----> shape:", X_train_val.shape)
print("Flattened X_test ----> shape:", X_test.shape)


Loaded X_train_val ----> shape: (589600, 32, 3)
Loaded X_test      ----> shape: (290400, 32, 3)
Loaded Y_train_val ----> shape: (589600, 32, 3)
Loaded Y_test      ----> shape: (290400, 32, 3)
Flattened X_train_val ----> shape: (589600, 96)
Flattened X_test ----> shape: (290400, 96)


# The ground truth is incorporated in the ['g', 'q', 'w', 'z', 't'] vector of boolean, taking the form
 
## [1, 0, 0, 0, 0] for gluons
 
## [0, 1, 0, 0, 0] for quarks
 
## [0, 0, 1, 0, 0] for Ws
 
## [0, 0, 0, 1, 0] for Zs
 
## [0, 0, 0, 0, 1] for tops

## This is what is called 'one-hot' encoding of a descrete label (typical of ground truth for classification problems)



## Define our Deep MLP model


In [11]:
# Chamfer Loss
K = keras.backend
def chamfer_loss(y_true, y_pred):           

    # flatten the batch 
    y_true_f = K.batch_flatten(y_true)
    y_pred_f = K.batch_flatten(y_pred)

    y_pred_mask_f = K.sigmoid(y_pred_f - 0.5)

    finalChamferDistanceSum = K.sum(y_pred_mask_f * y_true_f, axis=1, keepdims=True)  

    return K.mean(finalChamferDistanceSum)

In [12]:
# baseline keras model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, BatchNormalization
from tensorflow.keras import utils
#from tensorflow.keras import mixed_precision

print("X type=",type(X_train_val))
print("X shape=",X_train_val.shape)

# Define MLP input layer dimension ( NINPUT = neta*nphi )
NINPUT = len(X_train_val[1])
NOUTPUT = len(Y_train_val[1])

print("NINPUT = ",NINPUT)
print("NOUTPUT = ",NOUTPUT)

'''
# MLP architechture for 32 contituents
nhidden1 = int(NINPUT)
nhidden2 = int(NINPUT)
'''

'''
# MLP architechture for 16 contituents
nhidden1 = int(NINPUT)
nhidden2 = int(NINPUT)
'''


# MLP architechture for 8 contituents
#nhidden1 = int(NINPUT*2.3)
#nhidden2 = int(NINPUT*2.3)

# MLP architechture for 8 contituents
nhidden = int(NINPUT*1.75) 

#############################################################################################


# Define the input tensor shape
input  = Input(shape=(NINPUT,), name = 'input') 
hidden = BatchNormalization(name='batchnorm')(input)

# Define MLP with 2 hidden layers 
hidden = Dense(nhidden,  name = 'Dense1', activation='relu')(hidden)
hidden = Dense(nhidden,  name = 'Dense2', activation='relu')(hidden)
hidden = Dense(nhidden,  name = 'Dense3', activation='relu')(hidden)

output  = Dense(NOUTPUT,   name = 'DenseOut',  activation='softmax')(hidden)

# create the model
model = Model(inputs=input, outputs=output)
# Define the optimizer ( minimization algorithm )
#optim = SGD(learning_rate=0.01,decay=1e-6)
optim = Adam(learning_rate=0.0001)
#optim = Adam(learning_rate=0.0005)
#optim = Adam()

# compile the model
model.compile(optimizer=optim, loss=chamfer_loss, metrics=['categorical_accuracy'])


# print the model summary
model.summary()

X type= <class 'numpy.ndarray'>
X shape= (589600, 24)
NINPUT =  24
NOUTPUT =  5
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 24)]              0         
_________________________________________________________________
batchnorm (BatchNormalizatio (None, 24)                96        
_________________________________________________________________
Dense1 (Dense)               (None, 42)                1050      
_________________________________________________________________
Dense2 (Dense)               (None, 42)                1806      
_________________________________________________________________
Dense3 (Dense)               (None, 42)                1806      
_________________________________________________________________
DenseOut (Dense)             (None, 5)                 215       
Total params: 4,973
Trainable params: 4,925
N

## Run training
Here, we run the training.

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# early stopping callback
es = EarlyStopping(monitor='val_categorical_accuracy', patience=10)

# Learning rate scheduler 
lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.2, patience=10)

# model checkpoint callback
# this saves our model architecture + parameters into mlp_model.h5
chkp = ModelCheckpoint('MLP_model.h5', monitor='val_categorical_accuracy', 
                       verbose=0, save_best_only=True, 
                       save_weights_only=False, mode='auto', 
                       period=1)


# Train classifier
history = model.fit(  X_train_val, Y_train_val, 
                    epochs=300, 
                    batch_size=512, 
                    verbose=1,
                    callbacks=[lr, es, chkp], 
                    validation_split=0.3 )
                    
#                    callbacks=[early_stopping, model_checkpoint], 

# Set NN and output name
arch = 'MLP'
fname = arch+'_nconst_'+str(nmax)
print('Saving Model : ',fname)


## Save the model+ weights
model.save(fname+'.h5')

## Save the model weights in a separate file
model.save_weights('weights'+fname+'.h5')                    

Epoch 1/300
807/807 [==============================] - 1s 961us/step - loss: 0.4120 - categorical_accuracy: 0.1229 - val_loss: 0.3944 - val_categorical_accuracy: 0.0553
Epoch 2/300
807/807 [==============================] - 1s 907us/step - loss: 0.3916 - categorical_accuracy: 0.0515 - val_loss: 0.3904 - val_categorical_accuracy: 0.0498
Epoch 3/300
807/807 [==============================] - 1s 909us/step - loss: 0.3900 - categorical_accuracy: 0.0486 - val_loss: 0.3898 - val_categorical_accuracy: 0.0485
Epoch 4/300
807/807 [==============================] - 1s 922us/step - loss: 0.3896 - categorical_accuracy: 0.0478 - val_loss: 0.3895 - val_categorical_accuracy: 0.0479
Epoch 5/300
807/807 [==============================] - 1s 973us/step - loss: 0.3894 - categorical_accuracy: 0.0472 - val_loss: 0.3894 - val_categorical_accuracy: 0.0477
Epoch 6/300
807/807 [==============================] - 1s 935us/step - loss: 0.3892 - categorical_accuracy: 0.0466 - val_loss: 0.3893 - val_categorical_acc

## Plot performance
Here, we plot the history of the training and the performance in a ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

#plt.rcParams['axes.unicode_minus'] = False

# Plot loss vs epoch
plt.figure(figsize=(15,10))
ax = plt.subplot(2, 2, 1)
ax.plot(history.history['loss'], label='loss')
ax.plot(history.history['val_loss'], label='val loss')
ax.legend(loc="upper right")
ax.set_xlabel('epoch')
ax.set_ylabel('loss')

# Plot accuracy vs epoch
ax = plt.subplot(2, 2, 2)
ax.plot(history.history['categorical_accuracy'], label='categorical_accuracy')
ax.plot(history.history['val_categorical_accuracy'], label='val categorical accuracy')
ax.legend(loc="upper left")
ax.set_xlabel('epoch')
ax.set_ylabel('acc')

# Plot the ROC curves
labels = ['gluon', 'quark', 'W', 'Z', 'top']
fpr = {}
tpr = {}
auc1 = {}
precision = {}
recall = {}
NN = {}
NP = {}
TP = {}
FP = {}
TN = {}
FN = {}
tresholds = {}

ax = plt.subplot(2, 2, 3)
Y_predict = model.predict(X_test)

# Loop over classes(labels) to get metrics per class
for i, label in enumerate(labels):
    fpr[label], tpr[label], tresholds[label] = roc_curve(Y_test[:,i], Y_predict[:,i])
#    precision[label], recall[label], tresholds = precision_recall_curve(Y_test[:,i], Y_predict[:,i]) 
    print( np.unique(Y_test[:,i], return_counts=True) )
    _ , N = np.unique(Y_test[:,i], return_counts=True) # count the NEGATIVES and POSITIVES samples in your test set
    NN[label] = N[0]                   # number of NEGATIVES 
    NP[label] = N[1]                   # number of POSITIVES
    TP[label] = tpr[label]*NP[label]
    FP[label] = fpr[label]*NN[label] 
    TN[label] = NN[label] - FP[label]
    FN[label] = NP[label] - TP[label]

    auc1[label] = auc(fpr[label], tpr[label])
    ax.plot(tpr[label],fpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))
    
ax.semilogy()
ax.set_xlabel("sig. efficiency")
ax.set_ylabel("bkg. mistag rate")
ax.set_ylim(0.001,1)
#ax.set_grid(True)
ax.legend(loc='lower right')
#plt.savefig('%s/ROC.pdf'%(options.outputDir))

print('Tresholds=',tresholds)

# Plot DNN output 
ax = plt.subplot(2, 2, 4)
X = np.linspace(0.0, 1.0, 20)
hist={}
for i, name in enumerate(labels):
    hist[name] = ax.hist(Y_predict, bins=X, label=name ,histtype='step')

ax.semilogy()
ax.set_xlabel('DNN Output')
ax.legend(loc='lower left')
ax.legend(prop={'size': 10})

# Display plots
fig = plt.gcf()
plt.show()    

In [ ]:
from sklearn.metrics import accuracy_score

# Save plots
fig.savefig(fname+'.pdf')


# Save FPR for a given TPR value ( 30% , 50% & 80%)
with open('FPR@TPR_'+fname+'.csv', 'w') as file:
  file.write("model,label,treshold,tpr,fpr\n")
  for label in labels:
    for t in [0.3, 0.5, 0.8]:
      idx = np.argmax(tpr[label]>t)
      file.write( arch+','+label+','+str(t)+','+str(tpr[label][idx])+','+str(fpr[label][idx])+'\n' )
               
               
# Save ROC AUC for each label
with open('ROCAUC_'+fname+'.csv', 'w') as file:
  header = labels[0]+', '+labels[1]+', '+labels[2]+', '+labels[3]+', '+labels[4]+'\n'
  file.write(header)
  rocauc = str(auc1[labels[0]])+', '+str(auc1[labels[1]])+', '+str(auc1[labels[2]])+', '+str(auc1[labels[3]])+', '+str(auc1[labels[4]])
  file.write(rocauc)


# Save NN Accuracy for treshold of 0.5 for each label and the average over all classes
acc_avg = float(accuracy_score (np.argmax(Y_test,axis=1), np.argmax(Y_predict,axis=1)))
with open('ACCURACY_'+fname+'.csv', 'w') as file:
  header = labels[0]+', '+labels[1]+', '+labels[2]+', '+labels[3]+', '+labels[4]+', '+'acc_avg'+'\n'
  file.write(header)
  accuracy = ''
  for label in labels:  
    idx = np.argmax( tresholds[label] <= 0.5 )
    accuracy += str( (TP[label][idx]+TN[label][idx])/(NP[label]+NN[label]) )+', '
  accuracy += str(acc_avg) 
  file.write(accuracy)


'''
# Save confusion matrix ndarrays to .npz file
with open('CONF_MATRIX_'+fname+'.npz', 'wb') as file:
    vars = {}
    vars[arch]=np.array(1) # save model name
    for label in labels:
        vars['tresholds_'+label+'_'+arch] = tresholds[label]
        vars['TP_'+label+'_'+arch] = TP[label]
        vars['FP_'+label+'_'+arch] = FP[label]
        vars['TN_'+label+'_'+arch] = TN[label]
        vars['FN_'+label+'_'+arch] = FN[label]
        vars['TPR_'+arch] = tpr[label]
        vars['FPR_'+arch] = fpr[label]
        vars['NP_'+arch]= NP[label]
        vars['NN_'+arch]= NN[label]
        vars['auc_'+arch] = auc1[label] 
#        print(vars)
    np.savez(file, **vars)
'''


# Save a sample of events for HLS
njets=3000
print(X_test.shape)
np.save('x_test_8const.npy', X_test[0:njets,:])
np.save('y_test_8const.npy', Y_test[0:njets,:])
